## Classification (c)

In [1]:
# from neuralnet_update_PC import *
# from neuralnet_works_FC import *
from neuralnet import *
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt

config_c = {}
config_c['layer_specs'] = [3072, 64, 64, 10]
config_c['activation'] = 'tanh'
config_c['learning_rate'] = 0.005
config_c['batch_size'] = 128
config_c['epochs'] = 100  
config_c['early_stop'] = True 
config_c['early_stop_epoch'] = 5
config_c['L2_penalty'] = 0  
config_c['momentum'] = True  
config_c['momentum_gamma'] = 0.9  
# Create the model

#TODO
#momentum, early stop , expect accuracy around 37%

In [2]:
"""
Train your model here.
Implement batch SGD to train the model.
Implement Early Stopping.
Use config to set parameters for training like learning rate, momentum, etc.
"""
def train_e(model, x_train, y_train, x_valid, y_valid, config, patience=5):
    epochs = config_c['epochs']
    batch_size = config['batch_size']
    momentum =    config['momentum']
    momentum_gamma = config['momentum_gamma']
    L2_penalty = config['momentum_gamma']
    patience = config['early_stop_epoch']

    train_loss_record = []
    train_accuracy_record = []
    holdout_loss_record = []
    holdout_accuracy_record = []

    # How many times the validation loss has gone up in a row.
    cur_loss_up_sequence = 0

    print("num_mini-batches", len(x_train) / batch_size)

    for epoch in range(epochs):
        #model.zero_grad()
        batch_loss = []
        batch_accuracy = []
        for x, y in generate_minibatches(x_train, y_train, batch_size):

            # print("x:", x.shape)
            # print("y:", y.shape)
            # Forward Pass
            train_y, loss = model.forward(x, y)
            batch_loss.append(loss) 
            # Backward Pass
            model.backward()

            batch_accuracy.append(model.accuracy(x,y))

        model.updateweight() # update weight for each layer.

        #model.zero_grad() added inside of updateweight

        y_hat, loss = model.forward(x_train, y_train)
        acc = model.accuracy(x_train, y_train)
        
        # train_loss = np.mean(np.array(batch_loss))
        # train_accuracy = np.mean(np.array(batch_accuracy))

        train_loss = np.mean(np.array(loss))
        train_accuracy = np.mean(np.array(acc))
        
        holdout_loss = model.forward(x_valid, y_valid)[1]
        holdout_accuracy = model.accuracy(x_valid, y_valid)

        train_loss_record.append(train_loss)
        train_accuracy_record.append(train_accuracy)

        holdout_loss_record.append(holdout_loss)
        holdout_accuracy_record.append(holdout_accuracy)

        print(f' epoch: {epoch + 1}, train accuracy: {train_accuracy:.4f}, train_loss_norm:{train_loss:.4f}, '\
            f'valid_acc: {holdout_accuracy:.4f}, valid_loss_norm: {holdout_loss:.4f}')   
        print()

        # Save the best weights according to test set.
        if holdout_loss > max(holdout_loss_record):
            cur_loss_up_sequence += 1

            if cur_loss_up_sequence >= patience:
                print("earlystop")
                break
        else:
            cur_loss_up_sequence = 0
            # Save the best weights.
            model.save_load_weight(save=True)
    
    return train_loss_record, train_accuracy_record, holdout_loss_record, holdout_accuracy_record

model_c  = Neuralnetwork(config_c)

# Load the data
x_train, y_train, stats = load_data(path="./data",stats = None, mode="train")
x_test, y_test = load_data(path="./data",stats = stats, mode="test")

x_train, y_train, x_valid, y_valid = split_data(x_train,y_train)

train_loss_record, train_accuracy_record, holdout_loss_record, holdout_accuracy_record = train_e(model_c,x_train,y_train,x_valid,y_valid,config_c)

# Recall parameters with minimum validation loss
model_c.save_load_weight(save=False) # load data
test_accuracy = test(model_c, x_test, y_test) 
print(test_accuracy)


(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
num_mini-batches 351.5625
 epoch: 1, train accuracy: 0.1006, train_loss_norm:1.1081, valid_acc: 0.1010, valid_loss_norm: 1.1123

 epoch: 2, train accuracy: 0.1025, train_loss_norm:1.0924, valid_acc: 0.1044, valid_loss_norm: 1.0991

 epoch: 3, train accuracy: 0.1034, train_loss_norm:1.0841, valid_acc: 0.1042, valid_loss_norm: 1.0917

 epoch: 4, train accuracy: 0.1041, train_loss_norm:1.0785, valid_acc: 0.1042, valid_loss_norm: 1.0867

 epoch: 5, train accuracy: 0.1049, train_loss_norm:1.0743, valid_acc: 0.1046, valid_loss_norm: 1.0831

 epoch: 6, train accuracy: 0.1053, train_loss_norm:1.0709, valid_acc: 0.1042, valid_loss_norm: 1.0803

 epoch: 7, train accuracy: 0.1056, train_loss_norm:1.0681, valid_acc: 0.1044, valid_loss_norm: 1.0779

 epoch: 8, train accuracy: 0.1057, train_loss_norm:1.0658, valid_acc: 0.1044, valid_loss_norm: 1.0758

 epoch: 9, train accuracy: 0.1059, train_loss_norm:1.0637, valid_acc: 0.1046, valid_loss_norm: 1.0740

 epo


KeyboardInterrupt



In [ ]:

plt.figure(1)
plt.plot(np.arange(config_c['epochs']), train_loss_record, label='train')
plt.plot(np.arange(config_c['epochs']), holdout_loss_record, label='valid')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss vs epochs')
plt.legend()
plt.show()


plt.figure(2)
plt.plot(np.arange(config_c['epochs']), train_accuracy_record, label='train')
plt.plot(np.arange(config_c['epochs']), holdout_accuracy_record, label='valid')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy vs epochs')
plt.legend()
plt.savefig("plots/PA2_c_2.png")
plt.show()